<a href="https://colab.research.google.com/github/DiploDatos/AprendizajeProfundo/blob/master/0_set_up.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aprendizaje Profundo - Parte 0

Este notebook explica cómo instalar todos los prerequisitos y librería que van a necesitar para correr los tutoriales. Si pueden ejecutar las siguientes celdas, estás listo.

## Configuración del Entorno


### Instalar conda

Lo primero que necesitamos es instalar conda y crear un entorno para ejecutar el código. Hay dos formas de instalar conda: *Anaconda* y *Miniconda*. Cualquiera es útil para este curso. Simplemente seguí las instrucciones de acuerdo a tu sistema operativo.

https://docs.conda.io/projects/conda/en/latest/user-guide/install/index.html#regular-installation

### Crear un entorno con todas las librerías de Anaconda

    $ conda create --name deeplearning python=3.9.5 anaconda

No te olvides de activar el entorno

    $ conda activate deeplearning    

### Instalar PyTorch

Vamos a estar trabajando con [PyTorch](https://pytorch.org/) para armar y entrenar modelos de aprendizaje profundo. La librería es menos abstracta que otras posibilidades como [Keras](https://www.tensorflow.org/guide/keras) pero da más control al usuario y permite crear soluciones más flexibles.

Para instalar PyTorch recomendamos seguir la [documentación oficial](https://pytorch.org/get-started/locally/). En sus máquinas es mejor que instalen la versión con solo soporte para CPU (sin CUDA), pero en Nabucodonosor van a requerir una versión con soporte para GPU.

#### CPU

Instalar PyTorch para CPU:

    (deeplearning) $ conda install pytorch torchvision cpuonly -c pytorch
    
Luego solo resta chequear que la versión instalada sea  >= 1.10. Para ejecutar las siguientes líneas de código abrir python dentro del entorno (es suficiente con escribir <code>python</code> en la consola que estén usando).

In [ ]:
import torch

torch.__version__

'1.10.1+cu111'

#### GPU

La versión de GPU de PyTorch depende de la versión de CUDA instalada. Nabucodonosor tiene muchas instalaciones de CUDA en `/opt/cuda`. Tienen que agregar `nvcc` ([nvidia cuda compiler](https://docs.nvidia.com/cuda/cuda-compiler-driver-nvcc/index.html)) al `$PATH`. Por ejemplo, para CUDA 11.1, deben hacer lo siguiente:

    (deeplearning) $ export PATH=/opt/cuda/11.1/bin:$PATH

Eso se tiene que hacer cada vez que entren a nabucodonosor. Para evitar eso, pueden agregarlo a su `.bashrc`:

    (deeplearning) $ echo "export PATH=/opt/cuda/11.1/bin:$PATH" >> $HOME/.bashrc

Luego instalen la versión de PyTorch compatible con CUDA 11.1:

    (deeplearning) $ pip install torch==1.10.1+cu111 torchvision==0.11.2+cu111 torchaudio==0.10.1 -f https://download.pytorch.org/whl/torch_stable.html

Pueden controlar que la instalación funcione corriendo lo siguiente:

In [ ]:
torch.cuda.is_available()

True

Si `torch.cuda.is_available()==True`, PyTorch puede usar las GPUs.

#### Google Colab

En caso de que quieran usar PyTorch en Google Colab, es posible, pero primero deben controlar qué versión de `nvcc` está corriendo. Para eso corran el siguiente comando:

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


De acuerdo a lo que diga el comando anterior, tendrán que instalar los drivers apropiados con `pip`. Para más informacion vean la [documentación](https://pytorch.org/get-started/locally/) de cómo hacer.

### Instalar otras librerías

Necesitamos `gensim` para poder usar *word embeddings*, asi que necesitamos instalarlo. También necesitamos `mlflow` para hacer un seguimiento de los experimentos. Finalmente, `tqdm` es util para hacer barras de progreso para hacer un seguimiento del progreso.

    (deeplearning) $ conda install gensim mlflow tqdm -c conda-forge

Si tienen problemas importando `gensim` y tienen este error:

    ImportError: cannot import name 'open' from 'smart_open' (C:\ProgramData\Anaconda3\lib\site-packages\smart_open\__init__.py)

Intenten actualizando `smart_open`

    (deeplearning) $ conda update smart_open

## Descargar embeddings y datasets

### CIFAR10

El dataset que usaremos (CIFAR10) es parte del paquete de `torchvision`, el cual lo hace muy simple de descargar. Pueden leer los detalles [aquí](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#loading-and-normalizing-cifar10):

In [ ]:
import torchvision

torchvision.datasets.CIFAR10(root='./data', download=True);

### Glove Embeddings y IMDB reviews Dataset

Algunos ejemplos que vamos a correr para clasificación de texto usan Redes Neuronales Convolucionales y requieren los embeddings de Glove junto al dataset de IMDB reviews:

In [ ]:
!curl -L https://cs.famaf.unc.edu.ar/\~ccardellino/resources/diplodatos/glove.6B.50d.txt.gz -o ./data/glove.6B.50d.txt.gz
!curl -L https://cs.famaf.unc.edu.ar/\~ccardellino/resources/diplodatos/imdb_reviews.csv.gz -o ./data/imdb_reviews.csv.gz

### MeLi Challenge 2019 Dataset

Para el proyecto del curso vamos a usar el dataset del MeLi Challenge 2019, para clasificación automática de categorías de productos:

In [ ]:
!curl -L https://cs.famaf.unc.edu.ar/\~ccardellino/resources/diplodatos/meli-challenge-2019.tar.bz2 -o ./data/meli-challenge-2019.tar.bz2
!tar jxvf ./data/meli-challenge-2019.tar.bz2 -C ./data/


meli-challenge-2019/
meli-challenge-2019/spanish.test.jsonl.gz
meli-challenge-2019/portuguese.validation.jsonl.gz
meli-challenge-2019/portuguese.train.jsonl.gz
meli-challenge-2019/spanish.train.jsonl.gz
meli-challenge-2019/spanish_token_to_index.json.gz
meli-challenge-2019/portuguese_token_to_index.json.gz
meli-challenge-2019/spanish.validation.jsonl.gz
meli-challenge-2019/portuguese.test.jsonl.gz


## Using Nabucodonosor

### Tunneling y ssh

**Cómo correr un notebook en una máquina remota?** Usás una conección ssh con forwarding de puertos. De esta forma todo lo que va al puerto indicado en la máquina también va al localhost.

Es posible que muchos quieran usar el mismo puerto, asi que recomendamos que elijan un numero al azar antes de conectarse (menos el 16006). El puerto en el ssh debe ser el mismo en el que inicial el notebook.

```
$ ssh -L PORT:localhost:PORT USER@nabucodonosor.ccad.unc.edu.ar
$ conda activate deeplearning
(deeplearning) $ jupyter notebook --port PORT --no-browser
```

Ahora pueden usar el notebook como si estuvieran corriendo en su computadora.

### Evitar usar GPUs

Si todas las GPUs están siendo usadas, se puede forzar a PyTorch a usar la CPU. Para modelos simples esta es una muy buena opción.

La forma más facil es settear la variable de entorno `CUDA_VISIBLE_DEVICES=""` cuando corren sus comandos. Por ejemplo:

```
(deeplearning) $ CUDA_VISIBLE_DEVICES="" jupyter notebook --no-browser
(deeplearning) $ CUDA_VISIBLE_DEVICES="" exercise_1.py --experiment_name mlp_200
```